In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
import expand_sfac

In [ ]:
from matplotlib.colors import LightSource
from matplotlib import lines,patches
from numpy.linalg import norm
from mpl_toolkits.mplot3d.art3d import Line3DCollection, Poly3DCollection

In [ ]:
#import minghao's solutions
import math
mh_sol = [-0.16124279918824116, -0.3338971575276649, 0.11560245246284442, 0.23938638169753415]
u=mh_sol[0:2]
mu=mh_sol[2:4]
u=np.array(u)
mu=np.array(mu)
theta=math.atan(u[1]/u[0])
rot=np.array([[math.cos(theta),math.sin(theta)],[-math.sin(theta), math.cos(theta)]]);
uprime=np.dot(rot,u)
muprime=np.dot(rot,mu)
print(uprime)
print(muprime)
mh_sol_rot = [uprime[0], uprime[1],muprime[0],muprime[1]]

jlist = np.ones(12)
trillium = trillium_maker.make_psg_lattice(L);
nbonds=trillium.nbonds
norb=trillium.norbital

#psg IV
#notation : ux, uy, mu_x, mu_y
n_icouplings=4
tmat= np.zeros([4,nbonds+norb,n_icouplings])
for delta in [0,3] :
    tmat[delta,:,:]=0

tmat[1,0:nbonds,0]=1
tmat[2,0:nbonds,1]=1

tmat[1,nbonds:nbonds+norb,2]=1
tmat[2,nbonds:nbonds+norb,3]=1

trillium.add_ansatz(n_icouplings,tmat)
trillium.initH()
trillium.current_ansatz=0
trillium.print_corrfuncs(mh_sol,L,fname="cfuncs_z2_mh.npy")
trillium.print_corrfuncs(mh_sol_rot,L,fname="cfuncs_z2_mh_rot.npy")

In [ ]:
from psg_hamiltonian_class import psg_lattice
import scipy.optimize as opt
import scipy
import sys
import trillium_maker
L=30
np.set_printoptions(precision=16)
def callback_func(x,f) :
    #print("callback params:,",x.real)
    return 0

def callback_func2(x) :
    print("callback :,",x.real,opt_func(trillium_psg,x))
    #print("energies:",12*x[0]*(2*f[0]+x[0]),12*x[0]**2,trillium_psg.calc_energy(x))
    en=12*x[0]**2
    sysen=trillium.calc_energy(x)
    print("energies:",en,sysen/2,sysen+en)
    return 0
def cost_func_z2(psg,x) : 
    b,s= trillium.calc_exps(x)
    b1=np.mean(b[:,1])
    b2=np.mean(b[:,2])
    s1=np.mean(s[:,1])
    s2=np.mean(s[:,2])
    return [b1-x[0],b2-x[1],s1,s2]

jlist = np.ones(12)
trillium = trillium_maker.make_psg_lattice(L);
nbonds=trillium.nbonds
norb=trillium.norbital

#psg IV
#notation : ux, uy, mu_x, mu_y
n_icouplings=4
tmat= np.zeros([4,nbonds+norb,n_icouplings])
for delta in [0,3] :
    tmat[delta,:,:]=0

tmat[1,0:nbonds,0]=1
tmat[2,0:nbonds,1]=1

tmat[1,nbonds:nbonds+norb,2]=1
tmat[2,nbonds:nbonds+norb,3]=1

trillium.add_ansatz(n_icouplings,tmat)
trillium.initH()
trillium.current_ansatz=0
guess =[-0.16124279918824116, -0.3338971575276649, 0.11528008981876346, 0.23956892343616404]
#
#np.random.seed(10)
for trial in range(4) :
    solution = opt.root(lambda x: cost_func_z2(trillium,x), guess, callback=callback_func,method='broyden2',options={"maxiter":20})
    solution=solution.x
    trillium.print_corrfuncs(solution,L,fname="cfuncs_z2.npy")
    print("%16f"%(12*solution[0]**2))
    print("solution found")
    print(solution)
    break



In [ ]:
def cost_func_u1(psg,x) : 
    b,s= psg.calc_exps(x)
    b=np.mean(b[:,3])
    s=np.mean(s[0,3])
    return [b-x[0],s]

jlist = np.ones(12)
trillium=trillium_maker.make_psg_lattice(L);
nbonds=trillium.nbonds
norb=trillium.norbital

n_icouplings=2
tmat= np.zeros([4,nbonds+norb,n_icouplings])
for delta in [0,1,2,3] :
    tmat[delta,:,:]=0

tmat[3,0:nbonds,0]=1
tmat[3,nbonds:nbonds+norb,1]=1

trillium.add_ansatz(n_icouplings,tmat)
trillium.initH()
trillium.current_ansatz=0
mine=0
guess = [-0.3705359978761876,  0.2951004826472385]
for trial in range(2) :
    solution = opt.root(lambda x: cost_func_u1(trillium,x), guess, callback=callback_func,method='broyden2',options={"maxiter":20})
    sol_u1=solution.x
    trillium.print_corrfuncs(sol_u1,L,fname="cfunc_u1.npy")
    print("solution found")
    break
print(sol_u1)




In [ ]:
nbands=8; nsites= L**3; norb=4

u=0.25;
psgc=[[u,u,u], [0.5+u,0.5-u,1-u], [1-u,0.5+u,0.5-u], [0.5-u,1-u,0.5+u]]
psgc=np.array(psgc)

nbands=8
def process_cfuncs(fname,exp_fac) :
    cfuncs=np.load(fname)
    cfuncs= np.reshape(cfuncs,[L,L,L,nbands,nbands])
    cfuncR=np.fft.ifftn(cfuncs, axes=(0,1,2),norm="backward")
    sfacR=np.zeros([L,L,L,norb,norb],dtype=complex) 
    sfac_full=np.zeros([L,L,L],dtype=complex) 
    phases=np.zeros([L,L,L],dtype=complex) 
    
    for x in range(L) :
        for y in range(L) :
            for z in range(L) : 
                for m in range(norb) :
                    for n in range(norb) : 
                        sfacR[x,y,z,m,n]= (-3.0/8.0)*(np.abs(cfuncR[x,y,z,m,n])**2 + np.abs(cfuncR[x,y,z,m,norb+n])**2)
    sfacK= np.fft.fftn(sfacR,axes=(0,1,2),norm='backward')
    cfuncout=expand_sfac.expand(sfacK,L,norb,psgc,exp_fac);
    return cfuncout


In [ ]:
#expand Brillouin zone
cfunc_u1=process_cfuncs("cfunc_u1.npy",4)
cfunc_z2_mh=process_cfuncs("cfuncs_z2_mh.npy",4)
cfunc_z2_mh_rot=process_cfuncs("cfuncs_z2_mh_rot.npy",4)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
fig,ax=plt.subplots(ncols=3,figsize=(13,5))
data1= np.abs(cfunc_z2_mh)
data2= np.abs(cfunc_z2_mh_rot-cfunc_u1)
data3= np.abs(cfunc_z2_mh_rot-cfunc_z2_mh)
p1=ax[0].imshow(data1[:,:,0],origin='lower',cmap='Reds')
p2=ax[1].imshow(data2[:,:,0],origin='lower',cmap='Reds')
p3=ax[2].imshow(data3[:,:,0],origin='lower',cmap='Reds')
ax[0].set_title("U1(sb)-Z2(mh)")
ax[1].set_title("U1(sb)-Z2 rotated(mh)")
ax[2].set_title("Z2 rotated(mh)-Z2(mh)")
fig.colorbar(p1)
fig.colorbar(p2)
fig.colorbar(p3)

In [ ]:
#expanded Brillouin zone
cfunc_u1=process_cfuncs("cfunc_u1.npy",4)
cfunc_z2=process_cfuncs("cfuncs_z2.npy",4)
cfunc_z2_rot=process_cfuncs("cfuncs_z2_rot.npy",4)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
fig,ax=plt.subplots(ncols=3,figsize=(13,5))
cfuncoutS=cfuncout
#ax.imshow(np.abs(np.diagonal(cfuncoutS[:,:,:],axis1=0,axis2=1)),interpolation="spline16",origin='lower')
#p=ax.imshow(np.abs(cfuncoutS[:,:,30]),interpolation="spline16",origin='lower',cmap='Reds')
p1=ax[0].imshow(np.abs(cfuncout_z2_rot[:,:,0]),origin='lower',cmap='Reds')
p2=ax[1].imshow(np.abs(cfuncout_z2[:,:,0]),origin='lower',cmap='Reds')
p3=ax[2].imshow(np.abs(cfuncout_z2_rot[:,:,0]),origin='lower',cmap='Reds')
fig.colorbar(p1)
fig.colorbar(p2)
fig.colorbar(p3)

In [ ]:
%matplotlib ipympl
#triangles=[[alpha,beta,gamma]   ]
#print(triangles)
plt.close('all')
#ucell_col = Line3DCollection(ucell_lines, color='grey')
#bond_col = Line3DCollection(bonds,color='#2b8cbe',alpha=0.6)
ls=LightSource(azdeg=-80,altdeg=8)
#tri_col = Poly3DCollection(triangles, shade=True,lightsource=ls,facecolors='#ece7f2',alpha=0.5)
fig=plt.figure(figsize=(8,8))
ax=plt.axes(projection='3d')
farr = (data1>0.65) & (data1<0.67)
x=np.linspace(0,4*L,4*L)
x,y,z=np.meshgrid(x,x,x)
ax.set_xlim(40,80)
ax.set_ylim(40,80)
ax.set_zlim(40,80)
ax.scatter(x[farr],y[farr],z[farr],s=0.6)